# Beer Type Prediction

## Initialise Data and Environment

default in local docker container

training in AWS g3.4xlarge

In [1]:
import numpy as np
import pandas as pd
import os

# set parent folder for this notebook folder as working directory
os.chdir('..')

#### Load and transform data: 
brewery_name is dropped for now, because: 

- there are over 5,000 brewery names
    - hypothesis is that brewery_name is not going to help with model performance (this hypothesis is not tested with experiment in this report, but is suggested for future exercise)


- brewery_name may also cause typo problem in API, this will also increase complexity with API design

only 4 numeric features remains for modelling:

    review_aroma, review_appearance, review_palate, review_taste

In [2]:
# set loadstatus to false by default
loadstatus = False
traindata_path = 'data/processed/'

# check if training data is already stored, 
# if exist: load data and and set loadstatus to true

if os.path.exists(f'{traindata_path}X_train.npy'):
    X_train = np.load(f'{traindata_path}X_train.npy')
    X_val = np.load(f'{traindata_path}X_val.npy')
    X_test = np.load(f'{traindata_path}X_test.npy')
    y_train = np.load(f'{traindata_path}y_train.npy')
    y_val = np.load(f'{traindata_path}y_val.npy')
    y_test = np.load(f'{traindata_path}y_test.npy')
    
    loadstatus = True

In [3]:
loadstatus

False

In [4]:
from src.data.load_dataset import load_data

if loadstatus == False:
    raw_filepath = 'data/raw/beer_reviews.csv'
    cleaned_filepath = 'data/processed/beer_review_cleaned.csv'

    # load and cleanse data
    df_cleaned=load_data(raw_filepath, cleaned_filepath)
    
    # extract target label column
    target = df_cleaned.pop('beer_style')

loading raw data


/home/ec2-user/adsi_a2/src/data/load_dataset.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned.dropna(inplace=True)


In [5]:
# scale data with standard scaler, save the fitted scaler to folder
from src.data.load_dataset import scale_features
from sklearn.preprocessing import StandardScaler

if loadstatus == False:
    df_scaled = process_features(df_cleaned, sc)

In [6]:
# encode target with label encoder
from src.data.load_dataset import encode_label
from sklearn.preprocessing import LabelEncoder

if loadstatus == False:
    enc = LabelEncoder()
    enc_target = encode_label(target, enc)

In [7]:
from src.models.train_model import split_data

if loadstatus == False:
    X_train, X_val, X_test, y_train, y_val, y_test = split_data(df_scaled, enc_target)

### Convert data into tensor

In [8]:
from src.models.pytorch import PytorchDataset

train_dataset = PytorchDataset(X=X_train, y=y_train)
val_dataset = PytorchDataset(X=X_val, y=y_val)
test_dataset = PytorchDataset(X=X_test, y=y_test)

## Hyperparameter Tuning

- learning rate
- number of neurons in hidden layer
- add model layers

#### Define Training Function for Hyperparameter Tuning

In [11]:
from src.models.pytorch import PytorchMCxlayers
from src.models.pytorch import train_classification, test_classification
from src.models.pytorch import get_device
import torch
import torch.nn as nn
import torch.nn.functional as F
from matplotlib import pyplot as plt
import time

def train_model(lr=0.01, neus=256, epochs=3, batch_size=16):
    from src.models.train_model import model_test_case

    device = get_device()
    
    unique, counts = np.unique(y_train, return_counts=True)
    weights = counts / counts.sum()
    class_weights = torch.FloatTensor(weights).to(device)

    print(f'=========== learning rate: {lr}, hidden layer neurons: {neus} ============')
    model = PytorchMCxlayers(num_features=X_train.shape[1], hlayer_neurons=neus, device=device)
    
    model.to(device)

    criterion = nn.CrossEntropyLoss(weight=class_weights)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    N_EPOCHS = epochs
    BATCH_SIZE = batch_size

    train_losses = []
    train_accs = []
    valid_losses = []
    valid_accs = []

    for epoch in range(N_EPOCHS):
        tic = time.perf_counter()

        train_loss, train_acc = train_classification(train_dataset, model=model, criterion=criterion, optimizer=optimizer, batch_size=BATCH_SIZE, device=device)
        valid_loss, valid_acc = test_classification(val_dataset, model=model, criterion=criterion, batch_size=BATCH_SIZE, device=device)

        print(f'Epoch: {epoch}')
        print(f'\t(train)\t|\tLoss: {train_loss:.4f}\t|\tAcc: {train_acc * 100:.1f}%')
        print(f'\t(valid)\t|\tLoss: {valid_loss:.4f}\t|\tAcc: {valid_acc * 100:.1f}%')

        train_losses.append(train_loss)
        train_accs.append(train_acc)
        valid_losses.append(valid_loss)
        valid_accs.append(valid_acc)
        
        toc = time.perf_counter()
        print(f"Epoch took {toc - tic:0.4f} seconds")
        
    epoch_count = range(1, len(train_accs)+1)

    plt.plot(epoch_count, valid_losses, 'r--')
    plt.plot(epoch_count, valid_accs, 'b-')
    plt.legend(['Validation Loss', 'Validation Accuracy'])
    plt.xlabel('Epoch')
    plt.ylabel('Epoch Scores')
    plt.show()
    
    model_test_case(model=model, aroma=33, appearance=18, palate=46, taste=41, device=device)
    
    return model

#### Tune Learning Rate

In [14]:
from src.models.pytorch import train_classification, test_classification

rate = 25e-8
neus = 256
epochs = 5

model = train_model(lr=rate, neus=neus, epochs=epochs)

=========== learning rate: 2.5e-07, hidden layer neurons: 256 ============
Epoch: 0
	(train)	|	Loss: 0.2902	|	Acc: 4.9%
	(valid)	|	Loss: 0.2900	|	Acc: 7.4%
Epoch took 224.1515 seconds
Epoch: 1
	(train)	|	Loss: 0.2887	|	Acc: 7.3%
	(valid)	|	Loss: 0.2860	|	Acc: 7.4%
Epoch took 220.1671 seconds


KeyboardInterrupt: 

In [ ]:
from src.models.pytorch import train_classification, test_classification

rate = 5e-7
neus = 256
epochs = 10

model = train_model(lr=rate, neus=neus, epochs=epochs)

In [ ]:
print(model)

In [ ]:
from src.models.train_model import model_test_case
device = get_device()
model_test_case(model=model, aroma=3, appearance=1, palate=2, taste=2, device=device)


In [ ]:
from torch.utils.data import Dataset, DataLoader

y_pred_list = []

with torch.no_grad():
    model.eval()
    data = DataLoader(test_dataset, batch_size=1, collate_fn=None)
    for X_batch, _ in data:
        X_batch = X_batch.to(device)
        y_test_pred = model(X_batch)
        _, y_pred_tags = torch.max(y_test_pred, dim=-1)
        y_pred_list.append(y_pred_tags.cpu().numpy())

In [ ]:
unique, counts = np.unique(y_train, return_counts=True)
appear = dict(zip(unique, counts))
appear = pd.DataFrame(appear.items(), columns=['Beer', 'count'])
appear.sort_values('count', ascending=False)